<a href="https://colab.research.google.com/github/khalidjasir/ui-km-paper/blob/khalid-v1/KM_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install necessary library

In [2]:
%pip install numpy==2.0.2
%pip install scipy==1.15.3
%pip install pandas
%pip install telethon
%pip install pytz
%pip install emoji
%pip install indonlp
%pip install tqdm
%pip install swifter
%pip install fasttext
%pip install openpyxl
%pip install scikit-learn
%pip install gensim==4.3.3 --no-deps
%pip install python-dotenv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 12.3 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=06c6c127ec34465fdf09e797e339f0155ab09b602e2e997d99aa0743803daf66
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=c9d02ad4fb46ab236703c0768766804b1f2d1412508b7e3bfe70d754cabaa6bf
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Detect environment

In [3]:
import os
import sys
import subprocess

def detect_environment():
  try:
    import google.colab
    return 'colab'
  except ImportError:
    pass

  if 'CONDA_PREFIX' in os.environ or os.path.exists(os.path.join(sys.prefix, 'conda-meta')):
    return 'conda'

  return 'local'

env = detect_environment()

## Handle Key

In [4]:
if env == 'colab':
  from google.colab import userdata
  api_id = userdata.get('api_id')
  api_hash = userdata.get('api_hash')
elif env == 'conda':
  from dotenv import load_dotenv
  load_dotenv()
  api_id = os.getenv('api_id')
  api_hash = os.getenv('api_hash')
else:
  print('Unable to detect suitable environment!')

## Retrieve from telegram

In [ ]:
import csv
import time
from datetime import datetime
from telethon import TelegramClient
from pytz import timezone

session_name = 'my_session'
channel_input = 'https://t.me/diskusipajak'
wib_timezone = timezone('Asia/Jakarta')
start_date = wib_timezone.localize(datetime(2015, 1, 1))
end_date = wib_timezone.localize(datetime(2025, 4, 30))
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'telegram_messages_{timestamp}.csv'
batch_size = 10000

async def main():
  async with TelegramClient(session_name, api_id, api_hash) as client:
    channel = await client.get_entity(channel_input)

    buffer = []
    total_count = 0

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file, quoting=csv.QUOTE_ALL)
      writer.writerow([
        'id',
        'date',
        'text',
        'sender_id',
        'chat_id',
        'reply_to_msg_id',
        'views',
        'forwards',
        'buttons',
        'raw_text',
        'message_link'
      ])

    async for msg in client.iter_messages(channel, offset_date=start_date, reverse=True):
      if msg.date > end_date:
        break

      row = [
        msg.id,
        msg.date.astimezone(wib_timezone).strftime('%a %b %d %H:%M:%S %z %Y') if msg.date else "",
        msg.text.replace('\n', ' ').strip() if msg.text else "",
        msg.sender_id if msg.sender_id else '',
        getattr(msg, 'chat_id', getattr(msg.to_id, 'channel_id', '')),
        msg.reply_to_msg_id if msg.reply_to_msg_id else '',
        msg.views if msg.views is not None else '',
        msg.forwards if msg.forwards is not None else '',
        len(msg.buttons) if msg.buttons else 0,
        msg.raw_text.replace('\n', ' ').strip() if msg.raw_text else '',
        f'https://t.me/{channel.username}/{msg.id}'
      ]

      buffer.append(row)
      total_count += 1

      if total_count % batch_size == 0:
        with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file, quoting=csv.QUOTE_ALL)
          writer.writerows(buffer)

        buffer.clear()
        print(f'Written {total_count} messages. Sleeping for 10 seconds...')
        time.sleep(10)

    if buffer:
      with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        writer.writerows(buffer)
      print(f'Final batch written. Total messages: {total_count}')

    print(f'Done! Messages saved to {csv_filename}.')

await main()

Written 10000 messages. Sleeping for 10 seconds...
Written 20000 messages. Sleeping for 10 seconds...
Written 30000 messages. Sleeping for 10 seconds...
Written 40000 messages. Sleeping for 10 seconds...
Written 50000 messages. Sleeping for 10 seconds...
Written 60000 messages. Sleeping for 10 seconds...
Written 70000 messages. Sleeping for 10 seconds...
Written 80000 messages. Sleeping for 10 seconds...
Written 90000 messages. Sleeping for 10 seconds...
Written 100000 messages. Sleeping for 10 seconds...
Written 110000 messages. Sleeping for 10 seconds...
Written 120000 messages. Sleeping for 10 seconds...
Written 130000 messages. Sleeping for 10 seconds...
Written 140000 messages. Sleeping for 10 seconds...
Written 150000 messages. Sleeping for 10 seconds...
Written 160000 messages. Sleeping for 10 seconds...
Written 170000 messages. Sleeping for 10 seconds...
Written 180000 messages. Sleeping for 10 seconds...
Written 190000 messages. Sleeping for 10 seconds...
Written 200000 messag

## Move file for later processing

In [ ]:
if env == 'colab':
  file_path = '/content/drive/MyDrive/telegram-data/'
  !mkdir -p $file_path
  !mv $csv_filename $file_path
  print(f'Successfully moved {csv_filename} to {file_path}')
elif env == 'conda':
  file_path = './data/'
  !mkdir -p $file_path
  !mv $csv_filename $file_path
  print(f'Successfully moved {csv_filename} to {file_path}')
else:
  print('Unable to detect suitable environment!')

Successfully moved telegram_messages_20250519_042802.csv to /content/drive/MyDrive/telegram-data/


## Retrieve stored telegram messages

In [5]:
import pandas as pd

filename = 'telegram_messages_20250519_042802.csv'

try:
  if env == 'colab':
    file_path = f'/content/drive/MyDrive/telegram-data/{filename}'
    messages_df = df = pd.read_csv(file_path)
    print(f'Successfully read from {file_path} to dataframe')
  elif env == 'conda':
    file_path = f'./data/{filename}'
    messages_df = df = pd.read_csv(file_path)
    print(f'Successfully read from {file_path} to dataframe')
  else:
    print('Unable to detect suitable environment! Nothing loaded to dataframe')
except FileNotFoundError:
  messages_df = pd.DataFrame()
  print(f'File not found')
except Exception as e:
  messages_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(messages_df)

Successfully read from /content/drive/MyDrive/telegram-data/telegram_messages_20250519_042802.csv to dataframe


,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link
0,1,Thu Dec 31 13:10:16 +0700 2015,NaN,NaN,-1001014841061,NaN,NaN,NaN,0,NaN,https://t.me/diskusipajak/1
1,69,Thu Dec 31 14:45:57 +0700 2015,Grup membahas seputar permasalahan aplikasi ya...,8.580854e+07,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69
2,71,Thu Dec 31 14:54:33 +0700 2015,iya pak,8.472804e+07,-1001014841061,NaN,NaN,NaN,0,iya pak,https://t.me/diskusipajak/71
3,74,Thu Dec 31 14:56:16 +0700 2015,"pak untuk ebiling, itu per januari 2016 semua ...",8.472804e+07,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74
4,77,Thu Dec 31 14:59:04 +0700 2015,makasih pak,8.472804e+07,-1001014841061,NaN,NaN,NaN,0,makasih pak,https://t.me/diskusipajak/77
...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,Tue Apr 29 19:37:32 +0700 2025,punya ku kalo dicek data2nya udah oke semua. g...,1.275726e+09,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652
331202,533653,Tue Apr 29 19:42:58 +0700 2025,STP muncul karena telat bayar ppn masa Januari...,6.648129e+09,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653
331203,533654,Tue Apr 29 20:00:47 +0700 2025,"hehe, tadi pagi kami bahas ini disebelah",8.580854e+07,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654
331204,533655,Tue Apr 29 20:01:52 +0700 2025,Ijin tanya kawan Coretax masih lemot GK ya? L...,1.284664e+09,-1001014841061,NaN,NaN,NaN,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655


## Pre-processing functions

In [6]:
import unicodedata
import emoji
import re
import string
import indoNLP.preprocessing

def basic_clean(text):
  # Normalize Unicode (remove fancy fonts, underlines)
  text = unicodedata.normalize('NFKD', text)
  text = ''.join(c for c in text if not unicodedata.combining(c))

  # Translate emojis
  text = emoji.demojize(text, delimiters=(' ', ' '), language='id')

  # Remove HTML
  text = indoNLP.preprocessing.remove_html(text)

  # Remove URL
  text = indoNLP.preprocessing.remove_url(text)

  # Remove usernames
  text = re.sub(r'@\w+', '', text)

  # Remove RT
  text = re.sub(r'\brt\b', '', text)

  # Remove hashtag symbol but keep the word
  text = re.sub(r'#', '', text)

  # Remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()

  return text

def advanced_clean(text):
  # Remove word elongation
  text = indoNLP.preprocessing.replace_word_elongation(text)

  # Replace slang
  text = indoNLP.preprocessing.replace_slang(text)

  # Lowercase
  text = text.lower()

  # Remove punctuation
  text = text.translate(str.maketrans('', '', string.punctuation))

  # Remove digits
  text = re.sub(r'\d+', '', text)

  # Remove stopwords
  text = indoNLP.preprocessing.remove_stopwords(text)

  # Remove extra whitespace again (just in case)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

## Run pre-processing

In [ ]:
import swifter

messages_df.dropna(subset=['text', 'raw_text'], inplace=True)
messages_df['basic_clean'] = messages_df['text'].swifter.apply(basic_clean)
messages_df['advanced_clean'] = messages_df['basic_clean'].swifter.apply(advanced_clean)
messages_df.dropna(subset=['basic_clean', 'advanced_clean'], inplace=True)

display(messages_df)

## Filter out text that is shorter than 5 words

In [ ]:
telegram_df = messages_df[messages_df['text'].str.split().str.len() > 5].copy()

display(telegram_df)

## Identify language

In [ ]:
import fasttext

model_path = 'lid.176.bin'
model = fasttext.load_model(model_path)

texts = telegram_df['advanced_clean'].tolist()

predictions = model.predict(texts)

telegram_df['lang_detected'] = [label[0].replace('__label__', '') for label in predictions[0]]
telegram_df['lang_confidence'] = [float(score[0]) for score in predictions[1]]

lang_stats = telegram_df.groupby('lang_detected')['lang_confidence'].agg(
  count='count',
  min='min',
  q25=lambda x: x.quantile(0.25),
  mean='mean',
  median='median',
  q90=lambda x: x.quantile(0.9),
  max='max'
).reset_index().sort_values(by='count', ascending=False)

display(lang_stats)

## Filter text with Bahasa Indonesia

In [ ]:
filtered_df = telegram_df[
  (telegram_df['lang_detected'] == 'id')
].copy()

display(filtered_df)

## Find similarity with KNN

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['advanced_clean'])

nn = NearestNeighbors(metric='cosine', n_neighbors=10)
nn.fit(tfidf_matrix)

distances, indices = nn.kneighbors(tfidf_matrix)

threshold = 0.95
to_remove = set()

for i, (dists, neighbors) in enumerate(zip(distances, indices)):
  for dist, idx in zip(dists[1:], neighbors[1:]):
    sim = 1 - dist
    if sim >= threshold:
      to_remove.add(max(i, idx))

unique_df = filtered_df.drop(filtered_df.index[list(to_remove)]).reset_index(drop=True)

display(unique_df)

## Store to local file

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_filename = f'filtered_telegram_messages_{timestamp}.xlsx'
unique_df.to_excel(excel_filename, index=False)

if env == 'colab':
  file_path = f'/content/drive/MyDrive/telegram-data/'
  !mkdir -p $file_path
  !mv $excel_filename $file_path
  print(f'Successfully moved {excel_filename} to {file_path}')
elif env == 'conda':
  file_path = f'./data/'
  !mkdir -p $file_path
  !mv $excel_filename $file_path
  print(f'Successfully moved {excel_filename} to {file_path}')
else:
  print('Unable to detect suitable environment!')

## Retrieve filtered messeges

In [ ]:
import pandas as pd

filename = excel_filename # 'filtered_telegram_messages_YYYYMMDD_hhmmss.xlsx'

try:
  if env == 'colab':
    file_path = f'/content/drive/MyDrive/telegram-data/{filename}'
    unique_df = df = pd.read_csv(file_path)
    print(f'Successfully read {file_path}')
    print(f'Successfully read from {file_path} to dataframe')
  elif env == 'conda':
    file_path = f'./data/{filename}'
    unique_df = df = pd.read_csv(file_path)
    print(f'Successfully read {file_path}')
    print(f'Successfully read from {file_path} to dataframe')
  else:
    print('Unable to detect suitable environment!')
except FileNotFoundError:
  unique_df = pd.DataFrame()
  print(f'File not found')
except Exception as e:
  unique_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(unique_df)

## Run LDA topic modeling

In [ ]:
from collections import defaultdict
from gensim import corpora, models
from gensim.models import CoherenceModel

unique_df['date'] = pd.to_datetime(unique_df['date'], format='%a %b %d %H:%M:%S %z %Y')
unique_df['year'] = unique_df['date'].dt.year
unique_df['month'] = unique_df['date'].dt.month

monthly_yearly_topics = defaultdict(dict)

for (month, year), group in unique_df.groupby(['month', 'year']):
  tokenized_docs = group['advanced_clean'].apply(str.split).tolist()

  if len(tokenized_docs) < 5:
    continue

  dictionary = corpora.Dictionary(tokenized_docs)
  dictionary.filter_extremes(no_below=3, no_above=0.85)
  corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
  if not corpus:
    continue

  num_topics = 10

  lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
    alpha='auto',
    per_word_topics=True
  )

  topics = []
  for topic_id in range(num_topics):
    words_probs = lda_model.show_topic(topic_id, topn=7)
    words = [word for word, _ in words_probs]
    topics.append({
      'topic_id': topic_id,
      'top_words': words
    })

  coherence_model = CoherenceModel(
    model=lda_model,
    texts=tokenized_docs,
    dictionary=dictionary,
    coherence='c_v'
  )

  monthly_yearly_topics[month][year] = {
    'topics': topics,
    'coherence': coherence_model.get_coherence()
  }

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for target_month in range(1, 13):
  if target_month in monthly_yearly_topics:
    print(f'\n=== Comparing Topics for {month_names[target_month - 1]} ===\n')
    for year in sorted(monthly_yearly_topics[target_month]):
      print(f"\n--- {month_names[target_month - 1]} {year} (Coherence: {monthly_yearly_topics[target_month][year]['coherence']:.4f}) ---")
      for t in monthly_yearly_topics[target_month][year]['topics']:
        print(f"Topic {t['topic_id']}: {', '.join(t['top_words'])}")
  else:
    print(f'\n=== No data for {month_names[target_month - 1]} ===')